# CHROMOSOME
First we need to come up with a way to represent the problem with a chormosome: We assume that there are num_rows + 2 nodes placed on plain labeled start, end, and 1 to num_rows. Then ants should comeup with a path from the start node to the end node, possibly passing through some intermediary nodes on their way. If an ant passes through the i-th node, it means that in the corresponding solution to the set covering problem we want to include the i-th set. Since it doesn't matter what order the ants visit the nodes in, we assume that the do it lexicographically based on the indices. Thus a chromosome is a binary array of size num_rows, showing which nodes the ant took on its trip.

# INITIALIZATION
Now we need to initialize the variables, the initializataion is as follows:

number of ants and max population set empirically.

rho, alph, and beta set based on the criteria thought by professor BabaAli.

number of columns and number of rows set by the given problem.

pheromone values are initially one for every node, this is to avoid dividing by zero in *P_ij* function.

heuristic values are initially set to the number of elements each subset holds, as recommended by professor BabaAli, as well.

And finally a set of binary sets, that represent our solutions.

In [16]:
que = open("scp62.txt", "r")

#initialize the constants
num_ants = 1000
max_iterations = 10000
rho = 0.6
alpha = 8
beta = 20
num_rows = 0 #number of element of the universal world
num_columns = 0 #number of sets in the problem

num_rows, num_columns = map(int, que.readline().strip().split(" "))
print(num_rows, num_columns)
cost = []

i = 0
while i < num_columns:
    #tmp = que.readline().lstrip(" ").rstrip("\n").rstrip(" ").split(" ")
    temp = list(map(int, que.readline().lstrip(" ").rstrip("\n").rstrip(" ").split(" ")))
    #print(tmp, flush=True)
    #temp = map(int, tmp)
    #print(tuple(temp))
    cost = cost + temp
    i += len(temp)

print("COST=",cost, flush=True)

subset = [[] for _ in range(num_columns)]
for i in range(num_rows):
    cnt = int(que.readline().lstrip(" ").rstrip("\n").rstrip(" "))
    #print(cnt, flush=True)
    j = 0
    while j < cnt:
        temp = list(map(int, que.readline().lstrip(" ").rstrip("\n").rstrip(" ").split(" ")))
        #print(temp)
        for lis in temp:
            subset[lis-1].append(i)
        j += len(temp)
    
#print(subset[10])
pheromone_values = [1 for _ in range(num_columns)]
heuristic_values = [len(subset[i])/cost[i] for i in range(num_columns)]
solution = [[False for i in range(num_columns)] for _ in range(num_ants)]


200 1000
COST= [1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 7, 7, 7, 7, 7, 7, 7, 7, 8, 8, 8, 8, 8, 8, 9, 9, 9, 9, 9, 9, 9, 9, 9, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 14, 14, 14, 14, 14, 14, 14, 14, 15, 15, 15, 15, 15, 15, 15, 15, 15, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 17, 17, 17, 17, 17, 17, 17, 17, 17, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 19, 19, 19, 19, 19, 19, 19, 19, 20, 20, 20, 20, 20, 20, 20, 20, 21, 21, 21, 21, 21, 21, 21, 21, 21, 22, 22, 22, 22, 22, 22, 22, 22, 22, 23, 23, 23, 23, 23, 23, 23, 23, 23, 23, 23, 23, 23, 23, 23, 23, 24, 24, 24, 24, 24, 24, 24, 24, 24, 25, 25, 25, 25, 25, 25, 25, 26, 26, 26, 26, 26, 26, 26, 26, 26, 26, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27

# FITNESS FUNCTION
Now we need to come up with a fitness function:
in this function we should keep in mind two main factors:
1. number of elements of U that this solution covers
2. number of elements of S that this solution includes

The first one should be maximized and the second one should be minimized. Our goal is to find a solution that covers ALL the members of num_columns and has the minirhom number of elements. It is also good to note, false answers could also appear in the form of a solution that covers all the members of num_columns and has a minimal number of members from S, however this minimality cannot be extended to minimization.

In [17]:
def costval(ant_solution):
    cost_score = 0
    for i in range(num_rows):
        if ant_solution[i] == 1:
            cost_score += cost[i]
    return cost_score

def coverall(ant_solution):
    covered_elements = [0 for _ in range(num_rows)]
    coverallval = num_rows
    for i in range(num_rows):
        if ant_solution[i] == 1:
            for j in range(len(subset[i])):    
                covered_elements[subset[i][j]] = 1
    for i in range(num_rows):
        if covered_elements[i] != 1:
            coverallval -= 1
    return coverallval


def fitness(ant_solution):
    """Evaluate ant solution by counting the number of instance it covers, and its size

    Args:
        ant_solution (list): a binary chromosome that shows a solution.\

    Returns:
        int: a fitness value based on how well the solution is
    """
    # every set that is included in the solution adds a score of -2, further motivating the solutions to have fewer sets
    covered_elements = [0 for _ in range(num_rows)]
    fitness_score = 0
    for i in range(num_rows):
        if ant_solution[i] == 1:
            fitness_score -= cost[i]                          #having a set costs 2 points
            for j in range(len(subset[i])):    
                covered_elements[subset[i][j]] = 1
    
    # every covered element adds a score of 5
    covers_all = True
    for i in range(num_rows):
        if covered_elements[i] == 1:
            fitness_score += 1000                          #having an element is rewarded by 5 points
        else:
            covers_all = False

    # if the solution covers all of the memebers of the universal set, ...
    # it gets a score of 10 for every element instead of 5. (5 extra)
    if covers_all:
        fitness_score += (995 * num_columns)              #having all the elements is rewarded by 1000*5 points per element
    
    return fitness_score



# P<sub>ij</sub> FUNCTION
Since we are implementing the MMAS algorithm here we should also come up with a P<sub>ij</sub> function as described in the algorithm

In [18]:
import random

def P_ij(currNode, pheromone, heuristic, alpha, beta):
    """gets a solution and calculates possiblities for a probable next move, 
    the way this works is that if the ant is in the i-th node we assume that
    it can only go to the nodes i+1 till n. this way we can be sure it doesn't
    visit the same node twice. 

    Args:
        currNode (int): the node the ant is currently in.
        pheromone (list): pheromone values for all nodes, rhotable, passed by
        reference to save compute power
        heurisitc (list): heuristic values for each node, rhotable, passed by
        reference to save compute power
        alpha (int): control parameter alpha as in definition
        beta (int): control parameter beta as in definition


    Returns:
        int: the next node the ant will pass through
    """
    
    cumulative_probability = 0.0
    for i in range(currNode + 1, num_columns):
        cumulative_probability += ((pheromone[i]**alpha) + (heuristic[i]**beta))

    #print(cumulative_probability)
    nextNode = random.uniform(0.0, cumulative_probability)
    #print(nextNode)
    #print()
    cumulative_probability = 0.0
    for i in range(currNode + 1, num_columns):
        cumulative_probability += ((pheromone[i]**alpha) + (heuristic[i]**beta))
        if cumulative_probability >= nextNode:
            #print(i)
            return i
    #print("REACHED END")
    return num_columns #in case the for didn't return we should give the termination command



# INFERENCE
Now we should iterate:

In order to solve this problem we're using the Max-Min Ant System with slight tweaks to get a response quicker.

In [19]:
#Iteration
for t in range(max_iterations):
    for ant in range(num_ants):
        solution[ant] = [0 for _ in range(num_columns)]
        nextNode = -1
        while (True):
            nextNode = P_ij(nextNode, pheromone_values, heuristic_values, alpha, beta)
            if nextNode == num_columns:
                break
            solution[ant][nextNode] = True

    best_fitness = -99999
    best_solution = -1
    for ant in range(num_ants):
        fitness_solution = fitness(solution[ant])
        if fitness_solution >= best_fitness:
            best_fitness = fitness_solution
            best_solution = ant
    
    for i in range(num_columns):
        pheromone_values[i] = (1-rho) * pheromone_values[i]

    solution_size = 0
    for i in range(num_columns):
        if solution[best_solution][i] == 1:
            solution_size += 1

    for i in range(num_columns):
        if solution[best_solution][i] == 1:
            pheromone_values[i] += (rho) * (1/solution_size)

    #print(solution[best_solution])
    #print(costval(solution[best_solution]), coverall(solution[best_solution]))
    #print(pheromone_values)
    if (costval(solution[best_solution]) <= 360) and (coverall(solution[best_solution]) == num_rows):
        print(solution[best_solution])
        break

print("ALGORITHM FINISHED, IF NO SOLUTION WAS PRINTED, THEN A SOLUTION WAS NOT FOUND")

[True, True, True, True, True, True, 0, True, 0, True, True, 0, 0, 0, 0, 0, True, True, True, True, 0, True, 0, 0, 0, 0, 0, 0, True, True, True, True, True, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, True, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, True, True, True, 0, 0, 0, 0, 0, 0, 0, True, True, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, True, True, True, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, True, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

# Iterate for a fixed number of iterations
for t in range(max_iterations):
    # Evaluate each ant's solution
    fitness_values = [fitness(instance, ant_solution) for ant_solution in population]

    # Update pheromone matrix
    delta_pheromone = [[0.0 for j in range(num_columns)] for i in range(num_rows)]
    for i in range(num_rows):
        for j in range(num_columns):
            for ant_index in range(population_size):
                if population[ant_index][j] == 1:
                    delta_pheromone[i][j] += fitness_values[ant_index]
            delta_pheromone[i][j] /= num_columns
    for i in range(num_rows):
        for j in range(num_columns):
            pheromone_values[i][j] = (1 - rho) * pheromone_values[i][j] + rho * delta_pheromone[i][j]

    # Create new population of ants
    new_population = []
    for ant_index in range(population_size):
        ant_solution = [0 for j in range(num_columns)]
        for j in range(num_columns):
            if random.random() < q0:
                # Choose best column according to heuristic information
                best_column = max((eta[i][j] * pheromone_values[i][j], j) for i in range(num_rows))[1]
                ant_solution[best_column] = 1
            else:
                # Choose column probabilistically
                probabilities = [eta[i][j] * pheromone_values[i][j] ** alpha for i in range(num_rows)]
                probabilities_sum = sum(probabilities)
                probabilities = [p / probabilities_sum for p in probabilities]
                chosen_column = P_ij(probabilities)
                ant_solution[chosen_column] = 1
        new_population.append(ant_solution)
    population = new_population

# Choose best ant solution as final solution
best_ant_index = max(range(population_size), key=lambda i: fitness(instance, population[i]))
#return population[best_ant_index]

